In [ ]:
# https://auto.gluon.ai/stable/tutorials/multimodal/text_prediction/beginner_text.html#sentiment-analysis-task

In [ ]:
!pip3 install -U pip
!pip3 install -U setuptools wheel

In [ ]:
!pip3 install torch==1.13.1+cpu torchvision==0.14.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
!pip3 install torch==1.13.1+cu116 torchvision==0.14.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
!pip3 install autogluon

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

PATH = "/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/"

train = pd.read_csv(PATH+'data/train.csv')
test = pd.read_csv(PATH+'data/test.csv')
submission = pd.read_csv(PATH+'data/sample_submission.csv')

In [ ]:
train

,ID,prompt,answer,AI
0,690.0,What is the future of AI?,The future of Artificial Intelligence (AI) is ...,1.0
1,304.0,What is your biggest challenge in your career?,My biggest challenge in my career is balancing...,0.0
2,63.0,What is the tallest mountain in the world?,The tallest mountain in the world is Mount Eve...,1.0
3,894.0,What are the best 5 super cars?,McLaren 720S.\r\nFerrari 296 GTB.\r\nAudi R8.\...,0.0
4,796.0,What is deep learning?,a type of machine learning based on artificial...,0.0
...,...,...,...,...
743,283.0,What is the most important thing in a family?,The most important thing in a family for me is...,0.0
744,895.0,What are the best machine learning Models?,Linear regression.\r\nLogistic regression\r\nD...,0.0
745,963.0,How can I chop onions without crying?,The best thing to do is very simple: use a ver...,0.0
746,156.0,What is the best way to stay healthy?,The best way to stay healthy varies from perso...,0.0


## 1. 전처리

In [ ]:
train["text"]=train["prompt"]+train["answer"]
train=train[["text","AI"]]

In [ ]:
test["text"]=test["prompt"]+test["answer"]
test=test["text"]

In [ ]:
train["text"].isna().sum()

1

In [ ]:
test.isna().sum()

0

In [ ]:
train["AI"].isna().sum()

2

In [ ]:
train = train[train["AI"].isna()==False]

In [ ]:
train["AI"].isna().sum()

0

In [ ]:
train

,text,AI
0,What is the future of AI?The future of Artific...,1.0
1,What is your biggest challenge in your career?...,0.0
2,What is the tallest mountain in the world?The ...,1.0
3,What are the best 5 super cars?McLaren 720S.\r...,0.0
4,What is deep learning?a type of machine learni...,0.0
...,...,...
743,What is the most important thing in a family?T...,0.0
744,What are the best machine learning Models?Line...,0.0
745,How can I chop onions without crying?The best ...,0.0
746,What is the best way to stay healthy?The best ...,0.0


In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np

In [ ]:
import re
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
    
import emoji
def dataPreprocessing(x): 
    x = x.lower()
    x = removeHTML(x)
    x = emoji.demojize(x, delimiters=(" ", " "))
    x = re.sub("@\w+", '',x) # removing mentions (@)
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    x = re.sub(r"[^\w\s]", '',x) # to remove symbols
    x = re.sub("http\w+", '',x)
    x = re.sub("\s[a-z]\s", '',x)
    x = x.strip()
    return x

In [ ]:
train_text=train["text"].apply(lambda x: dataPreprocessing(x))

In [ ]:
test_text=test.apply(lambda x: dataPreprocessing(x))

In [ ]:
train_text

0      what is the future of aithe future of artifici...
1      what is your biggest challenge in your careerm...
2      what is the tallest mountain in the worldthe t...
3      what are the best  super carsmclaren\nferrari ...
4      what is deep learninga type of machine learnin...
                             ...                        
743    what is the most important thing infamilythe m...
744    what are the best machine learning modelslinea...
745    how canchop onions without cryingthe best thin...
746    what is the best way to stay healthythe best w...
747    do you think naruto is overratedi think naruto...
Name: text, Length: 746, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split

X = train_text
y = train['AI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
pd.DataFrame(X_train, columns=['text'])

,text
70,what is the name of the unit used to measure p...
164,what is the name of the largest underwater cav...
712,what is the name of the gas that makes up the ...
265,what is the name of the largest underwater cav...
250,what is the most important factor in makingdec...
...,...
71,what is your biggest strengthmy biggest streng...
106,what is the currency of australiaaustralian do...
270,what is the best way to stay positivethe best ...
435,what is the name of the largest underwater cav...


In [ ]:
pd.DataFrame(y_train, columns=['AI'])

,AI
70,1.0
164,0.0
712,0.0
265,0.0
250,0.0
...,...
71,0.0
106,0.0
270,0.0
435,0.0


In [ ]:
train_data = pd.concat([pd.DataFrame(X_train, columns=['text']), pd.DataFrame(y_train, columns=['AI'])], axis=1)
train_data

,text,AI
70,what is the name of the unit used to measure p...,1.0
164,what is the name of the largest underwater cav...,0.0
712,what is the name of the gas that makes up the ...,0.0
265,what is the name of the largest underwater cav...,0.0
250,what is the most important factor in makingdec...,0.0
...,...,...
71,what is your biggest strengthmy biggest streng...,0.0
106,what is the currency of australiaaustralian do...,0.0
270,what is the best way to stay positivethe best ...,0.0
435,what is the name of the largest underwater cav...,0.0


In [ ]:
test_data = pd.concat([pd.DataFrame(X_test, columns=['text']), pd.DataFrame(y_test, columns=['AI'])], axis=1)
test_data

,text,AI
208,what is the most important lesson you have lea...,0.0
259,what is your biggest challenge in lifemy bigge...,0.0
97,where to put code documentationsome use other ...,0.0
148,what is the key to success in the culinary art...,0.0
395,what is your take on the current state of the ...,1.0
...,...,...
554,what is the best way to stay organizedthe best...,0.0
69,what is the name of the largest living structu...,1.0
522,what is the meaning of lifethe meaning of life...,0.0
131,what is natural language processing nlpnlp is ...,0.0


## 2. 모델링

In [ ]:
label = 'AI'
eval_metric = 'acc'
time_limit = 3600 * 6

In [ ]:
from autogluon.multimodal import MultiModalPredictor
import uuid

model_path = PATH+f"model/{uuid.uuid4().hex}-automm_sst_6hours"
predictor = MultiModalPredictor(label=label, eval_metric=eval_metric, path=model_path)
# predictor = MultiModalPredictor(label=label, eval_metric=eval_metric)
predictor.fit(train_data, presets='best_quality', time_limit=time_limit)

INFO:lightning_fabric.utilities.seed:Global seed set to 123
AutoMM starts to create your model. ✨

- Model will be saved to "/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/model/dbd426300c5f42a7985fd6fb089d957f-automm_sst_6hours".

- Validation metric is "acc".

- To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/model/dbd426300c5f42a7985fd6fb089d957f-automm_sst_6hours
    ```

Enjoy your coffee, and let AutoMM do the job ☕☕☕ Learn more at https://auto.gluon.ai



/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit None Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 40: 'val_acc' was not in top 3
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
Start to fuse 3 checkpoints via the greedy soup algorithm.


Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

AutoMM has created your model 🎉🎉🎉

- To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/model/dbd426300c5f42a7985fd6fb089d957f-automm_sst_6hours")
    ```

- You can open a terminal and launch Tensorboard to visualize the training log:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/model/dbd426300c5f42a7985fd6fb089d957f-automm_sst_6hours
    ```

- If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub: https://github.com/autogluon/autogluon




## 3. 성능평가

In [ ]:
# 모델 성능평가
# predictor.leaderboard(silent = True)

In [ ]:
test_score = predictor.evaluate(test_data)
print(test_score)

Predicting: 0it [00:00, ?it/s]

{'acc': 0.86}


In [ ]:
# model_to_use = predictor.get_model_best()
model_pred = predictor.predict(pd.DataFrame(test_text, columns=['text']))

Predicting: 0it [00:00, ?it/s]

In [ ]:
submission['Category'] = model_pred

In [ ]:
submission = submission.astype({'Category':'int64'})

In [ ]:
submission

,Id,Category
0,710,1.0
1,487,0.0
2,136,1.0
3,44,1.0
4,627,0.0
...,...,...
244,702,0.0
245,500,0.0
246,818,0.0
247,584,0.0


In [ ]:
submission.to_csv(PATH+'submit/autoGluon_6hour_submission_230227.csv', index=False)